In [1]:
import pandas as pd


In [282]:
assays_long = pd.read_csv('assays_long.csv')

In [283]:
###Count up conditions using groupby:
numActive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Active').any())).values.sum()
numInactive = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (grouped_df['outcome'].str.contains('Inactive').any())).values.sum()

numBoth = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: ((grouped_df['outcome'].str.contains('Inactive').any()) & 
                        (grouped_df['outcome'].str.contains('Active').any()))).values.sum()

numNone = assays_long.groupby(['instance_id', 'pref_name']).apply( 
    lambda grouped_df: (~(grouped_df['outcome'].str.contains('Inactive').any()) & 
                        ~(grouped_df['outcome'].str.contains('Active').any()))).values.sum()

In [284]:
###Or count up conditions the boring way:
y=0
n=0
b=0
neither=0
instance_ids = assays_long.drop_duplicates(['instance_id', 'pref_name'])['instance_id']
pref_names = assays_long.drop_duplicates(['instance_id', 'pref_name'])['pref_name']
for i, name in zip(instance_ids, pref_names):
    mask = (assays_long['pref_name']==name) & (assays_long['instance_id']==i)
    group = assays_long[mask]
    outcomes = list(group['outcome'])
    if ('Active' in outcomes) and ('Inactive' in outcomes):
        b+=1
    elif ('Active' in outcomes):
        y+=1
    elif ('Inactive' in outcomes):
        n+=1
    else:
        neither+=1
        

In [285]:
#Verify both ways are the same:
print(f'Active: {y}')
print(f'Inactive: {n}')
print(f'Both: {b}')
print(f'None: {neither}')

Active: 146
Inactive: 27
Both: 8
None: 359


In [286]:
#Both ways are the same:
print(f'Active: {numActive-numBoth}')
print(f'Inactive: {numInactive-numBoth}')
print(f'Both: {numBoth}')
print(f'None: {numNone}')

Active: 146
Inactive: 27
Both: 8
None: 359


In [287]:
df = pd.DataFrame(columns=['results','activity', 'number'])
df.loc[0]=['Results', 'Active', numActive]
df.loc[1]=['Results', 'Inactive', numInactive]
df.loc[2]=['Results', 'Both', numBoth]
df.loc[3]=['Results', 'Unknown', numNone ]
df['proportion'] = df['number']/df['number'].sum()
df

,results,activity,number,proportion
0,Results,Active,154,0.276978
1,Results,Inactive,35,0.0629496
2,Results,Both,8,0.0143885
3,Results,Unknown,359,0.645683


In [288]:
chart_one = alt.Chart(df).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y='results',
    color=alt.condition(selector, 'activity', alt.value('lightgray')),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
)

chart_two = alt.Chart(df.iloc[:3]).transform_joinaggregate(
        Total='sum(number)'
    ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
    x=alt.X('PercentOfTotal:Q'),#,sort='descending'),
    y='results',
    color=alt.condition(selector, 'activity', alt.value('lightgray')),
    order=alt.Order('activity', sort='ascending')
).add_selection(
    selector
)

chart_one&chart_two


alt.VConcatChart(...)

In [274]:
####Interactive version:
# brush = alt.selection(type='interval', encodings=['x'])

# base = alt.Chart(df).transform_joinaggregate(
#         Total='sum(number)'
#     ).transform_calculate(PercentOfTotal="datum.number / datum.Total").mark_bar().encode(
#     #x='number',
#     x='PercentOfTotal:Q',
#     y='results',
#     color='activity')

# upper = base.encode(
#     alt.X('PercentOfTotal:Q', scale=alt.Scale(domain=brush)))

# lower = base.add_selection(brush)

# lower&upper 
